<div
  style="
    background-color: #f0f0f0;
    color:rgb(56, 56, 56);
    padding: 8px;
    display: flex;
    align-items: center;
    gap: 100px;
  "
>
  <img src="./images/brand.svg" style="max-height: 80px;">
  <strong>
    AI Saga: Data Science and Machine Learning</br>
    3.lab.1. Wisconsin Cancer Classification - Neural Networks
  </strong>
</div>

# week 3 - Neural Networks for Classification: Wisconsin Cancer Dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)
from sklearn.linear_model import LogisticRegression
from rich.table import Table
from rich.console import Console
import seaborn as sns


# nota  esto es momentaneo para el proximo lab planneo unificaar los inports  estoy a la  espera del merge requeste anteriror  para hacer un rebase
# y unificar los imports

## Configuración de semillas

- `torch.manual_seed(42):` Fija la semilla aleatoria de PyTorch
- `np.random.seed(42):` Fija la semilla aleatoria de NumP


In [ ]:
torch.manual_seed(42)
np.random.seed(42)

aqui  meramente cargamos dataset, con el `data = load_breast_cancer()`, luego extraemos las caracteristicas por tumor para X y y solo son etiquetas 0 maligno, 2 benigno

In [ ]:
data = load_breast_cancer()
X = data.data  # 30 características numéricas
y = data.target  # 0: maligno, 1: benigno
feature_names = data.feature_names

## Normalizacion 

- `StandardScaler():` Crea un normalizador de datos
- `fit_transform():` Normaliza las 30 características a media=0, desviación=1

aqui la normalizacion pues como es comun es por que el dataset tiene escalas bastante diferentes , esto hace que se tenga encuenta todas las caracteristica 


- test_size=0.2: 80% para entrenar, 20% para probar
- random_state=42: usa la semilla para división reproducible,  es decir cadaa ejcucion ser igual en la reparticion de datos
- stratify=y: Mantiene la misma proporción de benignos/malignos en ambos conjuntos


luego simpelemnte transformamos los arrays a tensores  por que la lib reria trabaja con tensores mas no con arrays, en PyTorch

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División estratificada para mantener proporciones de clases
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Conversión a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

## red neuronal

- class BreastCancerNet:  red neuronal  para cáncer de mama
- nn.Module: Hereda funcionalidades básicas de PyTorch


### Parámetros de entrada:

input_dim=30: Entrada de 30 características
hidden_dims=[64, 32, 16]: Capas ocultas de 64→32→16 neuronas
output_dim=2: Sabenigno/maligno
dropout_rate=0.3: 30% de neuronas se apagan para evitar un  sobreajuste

### loop 

Linear: Conexiones entre neuronas (30→64→32→16→2)
BatchNorm: Normaliza datos entre capas
ReLU: Función de activación (convierte negativos en 0)
Dropout: Apaga 30% de neuronas aleatoriamente


aqui en esta clase creamos  una red neuronal completa que recibe 30 características de un tumor y va a predecir si es benigno o maligno, usando el dropout  para evitar sobreajuste y mejorar el aprendizaje.

In [ ]:
class BreastCancerNet(nn.Module):
    def __init__(
        self, input_dim=30, hidden_dims=[64, 32, 16], output_dim=2, dropout_rate=0.3
    ):
        super(BreastCancerNet, self).__init__()

        layers = []
        prev_dim = input_dim

        # Capas ocultas con BatchNorm y Dropout
        for hidden_dim in hidden_dims:
            layers.extend(
                [
                    nn.Linear(prev_dim, hidden_dim),
                    nn.BatchNorm1d(hidden_dim),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                ]
            )
            prev_dim = hidden_dim

        # Capa de salida
        layers.append(nn.Linear(prev_dim, output_dim))

        self.model = nn.Sequential(*layers)
        self._initialize_weights()

    def forward(self, x):
        return self.model(x)

    def _initialize_weights(self):
        for layer in self.model:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)

## Configuracion del entrenamiento 

- ### model = BreastCancerNet()

Creamo  una instancia de la red neuronal que definimos antes
Inicializa todos los pesos y capas automáticamente

- ### criterion = nn.CrossEntropyLoss()

Define la función de pérdida para clasificación
Mide qué tan equivocadas están las predicciones del modelo

- ### optimizer = optim.Adam(...)

- Adam: Algoritmo que actualiza los pesos del modelo
- lr=0.001: Velocidad de aprendizaje 
- weight_decay=1e-4: Regularización para evitar sobreajuste


en resumen el model sirve como el cerebro es decir aqui va aprender , el criterion sera el que evalue si los resultados son correctos o no, y el optimizer hara las mejoras en base a los resultados 

In [ ]:
model = BreastCancerNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

## Loop para el aprendizaje 


### varaibles


- epochs = 500: vera los datos 500 veces
- loss_history: lista para guardar errores en cada época
- accuracy_history: lista para guardar precisión cada 10 épocas


### Ciclo for

- Forward: los datos pasan por la red  y basicamente  obtienen las predicciones
- Loss: compara predicciones con respuestas correctas
- Backward: calcula cómo ajustar cada peso
- Step: aplica los ajustes a los pesos

### Condicional


basicamente el condicional dentro del ciiclo `if (epoch + 1) % 10 == 0:` lo que hace es que cada 10 epocas evalua que tan bien va el aprendizaje , en ese momento activa las isguientes funciones:

- model.eval(): Activa todas las neuronas para la prueba
- torch.no_grad(): sirve para no guardar informaciona dicionbal usi no solo resultados lo que ahorra memoria


en conclusion para este apartado se entrena la red neuronal con 500 iteraciones, donde se va aprendiendoo sobre la marcha me explico va aprendindo de lso errores y se va ajustando en los pesos para mejorar las predicciones, guardando cada progresos para el analisi posterior al realizado




In [ ]:
epochs = 500
loss_history = []
accuracy_history = []

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass
    loss.backward()
    optimizer.step()

    loss_history.append(loss.item())

    # Calcular accuracy cada 10 épocas
    if (epoch + 1) % 10 == 0:
        model.eval()
        with torch.no_grad():
            train_outputs = model(X_train_tensor)
            _, predicted = torch.max(train_outputs, 1)
            accuracy = (predicted == y_train_tensor).float().mean()
            accuracy_history.append(accuracy.item())

## Evaluacion final del modelo 

- model.eval(): Pone el modelo en modo evaluación es deecir activa todas las neuronas
- torch.max(): benigno/maligno, toma la más alta como predicción
- numpy(): Convierte tensores a arrays para usar con sklearn

### Calculo de metricas:

- Accuracy: da el porcentaje  de predicciones correctas totales
- Precision: de los que dijo "maligno",nos dice  ¿cuántos realmente lo eran?
- Recall: de todos los malignos reales, ¿cuántos detectó?


### F1: Promedio de precision y recall

- Train: funcion para ver que tambien se  meoriza  los datos de entrenamiento
- Test: Qué tan bien se generaliza a datos nuevos 


en conclusion en este fragmaento evaluamos el rendimeinto fiinal de la red neuronal que entrenamos, esto pues calacualdo multiples metricas tento en datoos de eentrenamiento como los de  prueba lo que nos permite saber qu tan bien clasificados estan los tumoeres


In [ ]:
model.eval()
with torch.no_grad():
    # Predicciones de entrenamiento
    train_outputs = model(X_train_tensor)
    _, y_train_pred = torch.max(train_outputs, 1)
    y_train_pred = y_train_pred.numpy()

    # Predicciones de prueba
    test_outputs = model(X_test_tensor)
    _, y_test_pred = torch.max(test_outputs, 1)
    y_test_pred = y_test_pred.numpy()

# Cálculo de métricas
acc_train_nn = accuracy_score(y_train, y_train_pred)
acc_test_nn = accuracy_score(y_test, y_test_pred)
prec_train_nn = precision_score(y_train, y_train_pred)
prec_test_nn = precision_score(y_test, y_test_pred)
recall_train_nn = recall_score(y_train, y_train_pred)
recall_test_nn = recall_score(y_test, y_test_pred)
f1_train_nn = f1_score(y_train, y_train_pred)
f1_test_nn = f1_score(y_test, y_test_pred)

# Resultados previos


- acc_log_all = 0.9737     
- acc_log_2feat = 0.9825    

estas variables implemnte guardan los resultados anterirores paaracomparar con la red neuronal en este caso pues en el anterior resultado se evaluo con las mejores caracteristeicas pues se haran uso de eestas 2 no mas

### Caracteristicas 

aqui meramente  se buscan en lso indices las 2 caracteristicas especificas

- worst_radius_idx: Radio del tumor en su peor zona
- worst_fractal_idx: Complejidad de la forma del tumor

luego extraemos esas mismas 2 caracteristicas tanto del entrenamiento como de la prueba 


 - X_test_2feat
- X_train_2feat



In [ ]:
# Resultados anteriores establecidos
acc_log_all = 0.9737
acc_log_2feat = 0.9825

# Preparamos datos para red neuronal con 2 características
worst_radius_idx = np.where(feature_names == "worst radius")[0][0]
worst_fractal_idx = np.where(feature_names == "worst fractal dimension")[0][0]
X_train_2feat = X_train[:, [worst_radius_idx, worst_fractal_idx]]
X_test_2feat = X_test[:, [worst_radius_idx, worst_fractal_idx]]

## Red neuranal simplificada para las 2 caracteristicas


es parecida al modelo de la red que configuramos anteriormente pero adaptado para hacer la comparacion , en este caso se cambian estos datos :

- Arquitectura: 2 → 16 → 16 → 2 
- dropout al 0.2 porque hay menos datos

luego en la parte del entrenameinto neuronal como se muestra en el ccomntario del codigo de adapta para lo siguiente:

- Convierte los datos de 2 características a tensores
- Crea modelo, función de pérdida y optimizador específicos
- lr=0.01: Velocidad de aprendizaje más alta 
- luego en el loop se reduce de 500 a 200 epoocas a menos caracteristicas menos iteraciones, entonces esto simplifica las cosas para la red

#### evaluacion final 

`acc_nn_2feat = accuracy_score(y_test, y_test_pred_2feat.numpy())` aqui basicamnete se calcula accuracy de la red neuronal con solo  las 2 características que habiamos medido para comparar con los resultados de la regresion logistica  implemntada 


en conclusiion este fragmento nos permite simplificar la red para evaluarla junto con los resultados de la regresion para coparar si esta red neuronal mas simple es mejor o puede ponerse a la par con la regresion logistica usando solo las 2 varible propuestas





In [ ]:
class SimpleNet(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=16, output_dim=2, dropout_rate=0.2):
        super(SimpleNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        return self.model(x)


# Entrenamiento de red neuronal con 2 características
X_train_2feat_tensor = torch.tensor(X_train_2feat, dtype=torch.float32)
X_test_2feat_tensor = torch.tensor(X_test_2feat, dtype=torch.float32)

model_2feat = SimpleNet()
criterion_2feat = nn.CrossEntropyLoss()
optimizer_2feat = optim.Adam(model_2feat.parameters(), lr=0.01)

# Entrenamiento rápido (200 épocas)
for epoch in range(200):
    model_2feat.train()
    optimizer_2feat.zero_grad()
    outputs = model_2feat(X_train_2feat_tensor)
    loss = criterion_2feat(outputs, y_train_tensor)
    loss.backward()
    optimizer_2feat.step()

# Evaluación red neuronal 2 características
model_2feat.eval()
with torch.no_grad():
    test_outputs_2feat = model_2feat(X_test_2feat_tensor)
    _, y_test_pred_2feat = torch.max(test_outputs_2feat, 1)
    acc_nn_2feat = accuracy_score(y_test, y_test_pred_2feat.numpy())

# Vizaulizacion de los resultados

aqui basicamente hago los graficos para anlizar resultadoos en este caso se realiza un total de 3 graficos para demostrar el estudio estos fueron: 

- Grafico 1: CURVA DE PÉRDIDA


Eje X: epocas (1 a 500)
Eje Y: error del modelo en cada época

aqui podemos ver si el modelo está aprendiendo la perdida en la grafica deberia bajar tambien funciona para detectar si se "atasca" o aprende demasiado rápido


- Grafico 2:  ACCURACY DE ENTRENAMIENTO


Eje X: Épocas 10- 500
Eje Y: % de predicciones correctas
Tendencia esperada: Línea que sube (más accuracy = mejor)


- Grafico 3:  MATRIZ DE CONFUSIÓN


Cuadro 2x2 que compara predicciones vs realidad:


In [ ]:
# Curva de entrenamiento
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(loss_history)
plt.title("Curva de Pérdida Durante Entrenamiento")
plt.xlabel("epoca")
plt.ylabel("CrossEntropy Loss")
plt.grid(True)

plt.subplot(1, 3, 2)
epochs_acc = range(10, epochs + 1, 10)
plt.plot(epochs_acc, accuracy_history)
plt.title("Accuracy Durante Entrenamiento")
plt.xlabel("Época")
plt.ylabel("Accuracy")
plt.grid(True)

# Matriz de confusión
plt.subplot(1, 3, 3)
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["Maligno", "Benigno"],
    yticklabels=["Maligno", "Benigno"],
)
plt.title("Matriz de Confusión - Red Neuronal")
plt.xlabel("Predicción")
plt.ylabel("Realidad")

plt.tight_layout()
plt.show()

## Analisis



### curva de perdida:


Empieza alto (~1.0) y baja rápidamente luego se estabiliza alrededor de 0.05 después de 100 epocas y ahi la curva ya no presenta mas oscilaciones, lo que se traduce en  que el modelo aprendio correctamente que no hubo un sobreajuste y la convergencia fue rapida pues no se necesito de muchas epocas


###  Accuary de entrenamiento 


Empieza en 50%, subee rápidamente a 100% en las primeras 50 épocas y se mantiene estable cerca del 100% qu es basicamente una alta precisión mantenida durante todo el entrenamiento


### matriz de confusion 


Maligno - Maligno: 41 canceres detectados
Maligno - Benigno: 1 cancer no detectado 
Benigno - Maligno: 4  falsa alarma
Benigno - Benigno: 68 sanos correctos

Métricas calculadas:

Accuracy: (41+68)/114 = 95.6%
Solo 5 errores de 114 casos


el modelo que se realizao en teoria funciona bien pero ele no detectar un cancer hablkando en situaciones reales puede resultar en un error grave por ende se puede ajustar el modelo , para qeu los detecte aunque puede que lso resultado cambien 

##  Tabla comparativa 

- results_table = solo creamos la tabla usando rich 
- results_table.add_column  = son las  columnas a representar en esta tabla Modelo,Características,Precisión. Cada una configurada con posicion y color

- results_table.add_row = agrega 4 filas las cuales  usan klas varibles ya definidas anterirormente para  representyar 
- Regresión Logística: 30 features (97.37%) y 2 features (98.25%)
- Red Neuronal: 30 features (95.61%) y 2 features (93.86%)

La regresión logística funciono mejor que las redes neuronales en este dataset, especialmente con pocas características.



In [ ]:
results_table = Table(
    title="Comparación: Regresión Logística vs Redes Neuronales",
    show_header=True,
    header_style="bold",
)

results_table.add_column("Modelo", justify="left", style="cyan")
results_table.add_column("Características", justify="center", style="yellow")
results_table.add_column("Precisión", justify="right", style="green")

# Resultados anteriores (baseline)
results_table.add_row("Regresión Logistica", "30 caracteristicas", f"{acc_log_all:.4f}")

results_table.add_row(
    "Regresion Logoistica",
    "worst radius...\nworst fractal dimensions",
    f"{acc_log_2feat:.4f}",
)

# Nuevos resultados (redes neuronales)
results_table.add_row("Red Neuronal", "30 características", f"{acc_test_nn:.4f}")

results_table.add_row(
    "Red Neuronal", "worst radius...\nworst fractal dimensions", f"{acc_nn_2feat:.4f}"
)

console = Console()
console.print(results_table)

In [ ]:
mejor_modelo = max(
    [
        ("Reg. Logística (30 feat)", acc_log_all),
        ("Reg. Logística (2 feat)", acc_log_2feat),
        ("Red Neuronal (30 feat)", acc_test_nn),
        ("Red Neuronal (2 feat)", acc_nn_2feat),
    ],
    key=lambda x: x[1],
)

print(
    f"MEJOR MODELO: {mejor_modelo[0]} con {mejor_modelo[1]:.4f} ({mejor_modelo[1]*100:.2f}%)"
)

## Analisis general 

en resumen se implemntaron e implementaron dos arquitecturas de redes neuronales una compleja de 30→64→32→16→2 y otra simple de 2→16→16→2 con una tecnica llamda  BatchNorm, Dropout y optimizador Adam, entrenadas durante 500 y 200 épocas respectivamente. Los resultados muestran que la regresión logística 98.25% con 2 características superó a las redes neuronales 95.6% con 30 características y 93.9% con 2, lo qeu nos dice  que modelos simples pueden ser más efectivos en datasets pequeños como este , visualizaciones completas y métricas exhaustivas, aunque presenta 1 falso negativo crítico para aplicaciones médicas, dandom=nos a entender que la simplicidad puede ayudar a una mejor genralizacion que un caomplejidad mas alla como la de una red y su arquitecturta con datos limitado como este dataset
